# Drug plates

In [ ]:
library(ggplot2)
library(extrafont)
library(stringr)
library(heatmaply)
library(ggrepel)
library(reticulate)
library(gridExtra)
library(robustbase)
library(randomForest)
library(reshape2)

In [ ]:
# Load external Python functions
source_python("reticulate_functions.py")
# Load extra fonts
ttf_import(paths = "/tmp/.fonts/")
loadfonts()
# Set this to true for faster compilation or false for more precise results and all outputs
TEST_MODE = T

In [ ]:
# Number of permutations used for empirical p-value computations
PERM_NB_ITER = ifelse(TEST_MODE, 20, 2000)
# Number of trees used in random forest classifier
RF_NB_TREES = ifelse(TEST_MODE, 50, 1000)

In [ ]:
# Customize ggplot appearance

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
cust_pal = colorRampPalette(c("#008CAD", "#40B9D4", "#D4ECF2", "#F8B100", "#C00000", "#2D0000"))
yolla_pal = colorRampPalette(c('#FC7070', '#C00000', '#2D0000'))
scale_fill_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)

## NK92

In [ ]:
# Fix random number generation
set.seed(38)

In [ ]:
NK = read.csv("Rsc/compiled_NKDrug_PerfPosition_Image.csv", header=T)

NB: Image measurements are matching previous run of CellProfiler (which did not include unstacked granule positional information).
```
NK = read.csv("Rsc/allImages_NK_Drugs.csv", header=T)
NK2 = read.csv("Rsc/compiled_NKDrug_PerfPosition_Image.csv", header=T)
ordNK = order(NK$Metadata_Well, NK$Metadata_Field)
ordNK2 = order(NK2$Metadata_Well, NK2$Metadata_Field)
dft = data.frame(x = NK$Texture_SumEntropy_CorrDNA_6_01[ordNK],
                 y = NK2$Texture_SumEntropy_CorrDNA_6_01[ordNK2])
ggplot(dft, aes(x = x, y = y)) + geom_point() + xlab("Texture first run") + ylab("Texture second run")
```
However slightly more cells were identified in the second run - despite the segmentation thresholds being the same. Ignoring this for now.

In [ ]:
Layout = read.csv("Rsc/DrugPlateLayout.csv", header=T)
Layout$Well <- paste0(Layout$Row, str_pad(Layout$Column, 2, pad = "0"))

In [ ]:
NK$Drug = sapply(NK$Metadata_Well, function(x) Layout$Drug[x == Layout$Well])
levels(NK$Drug)[levels(NK$Drug)=="Y27"] <- "Y-27632"
levels(NK$Drug)[levels(NK$Drug)=="Jasplaknolide"] <- "Jasplakinolide"
NK$Concentration = sapply(NK$Metadata_Well, function(x) Layout$Concentration[x == Layout$Well])

In [ ]:
gpNK = ggplot(NK[!is.na(NK$Count_FilteredNucleus),]) + geom_histogram(aes(Count_FilteredNucleus, fill = as.factor(Metadata_Row)), binwidth=2) +
       scale_fill_discrete(name="Row")
gpNK

if(!TEST_MODE){
    ggsave(filename = "Fig/NK_Drug_count.pdf", plot = gpNK)
}

In [ ]:
gpNK = ggplot(NK[!is.na(NK$Mean_FilterCytoplasm_AreaShape_Area),]) + geom_histogram(aes(Mean_FilterCytoplasm_AreaShape_Area, 
                                                                                    fill = as.factor(Metadata_Row)), binwidth=100) +
       scale_fill_discrete(name="Row")
gpNK
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_Drug_area.pdf", plot = gpNK)
}

### Filtering

In [ ]:
FILT_MAX_INT_DNA = 0.05 # Remove empty images and small DNA precipitations
FILT_MIN_CELLS = 4 # 8 seems safe from distribution and images, 3 seems in poor shape
FILT_NB_MAX_NA_IMAGE = 10
FILT_MAX_CORR = 0.6 # Keep uncorrelated variables
dimUMAP = 3

In [ ]:
ftToKeep = 1:dim(NK)[2]
# Make sure that the fields are numeric
ftToKeep <- ftToKeep[which(sapply(NK[,ftToKeep], class) == "numeric")]
# Remove execution time and count features
ftToKeep <- ftToKeep[grep("(Execution)|(Count)|(Concentration)", colnames(NK)[ftToKeep], invert = T)]

In [ ]:
# Store all remaining features before filtering for downstream comparison to selected set
preFiltFt = colnames(NK)[ftToKeep]

In [ ]:
# Remove wells with low max DNA intensity
fieldToKeep <- which(NK$ImageQuality_MaxIntensity_DNA >= FILT_MAX_INT_DNA)
# Remove wells with low cell count
fieldToKeep <- fieldToKeep[NK[fieldToKeep,]$Count_FilteredNucleus >= FILT_MIN_CELLS]

In [ ]:
# Few bad quality pictures are generating a lot of missing values and are removed
fieldToKeep <- fieldToKeep[rowSums(is.na(NK[fieldToKeep,ftToKeep])) < FILT_NB_MAX_NA_IMAGE]
# We exclude drugs for which more than half of the images were filtered
drugToRemove <- levels(NK$Drug)[table(NK$Drug[fieldToKeep]) < 180]
fieldToKeep <- fieldToKeep[!NK$Drug[fieldToKeep] %in% drugToRemove]

In [ ]:
# Remove remaining features with missing values
ftToKeep <- ftToKeep[colSums(is.na(NK[fieldToKeep,ftToKeep])) == 0] 
# Remove constant columns
indWT = NK[fieldToKeep,]$Drug == "WT"
ftToKeep <- ftToKeep[sapply(NK[fieldToKeep,ftToKeep], function(x) mad(x) != 0)]
ftToKeep <- ftToKeep[sapply(NK[fieldToKeep[indWT],ftToKeep], function(x) mad(x) != 0)]

In [ ]:
dim(NK)
dim(NK[fieldToKeep, ftToKeep])

Now, we don't do a full profiling but we want to explore values pertaining to perforin granule positioning, so we don't transform the data.

## Granule count per image

In [ ]:
subNK = NK[fieldToKeep,]
subNK$Drug = droplevels(subNK$Drug)
granuleCountWT = subNK[subNK$Drug == "WT",grep("Count_PerfGranules", names(subNK))]
granuleCountWT$imageID <- as.numeric(rownames(granuleCountWT))
granuleCountWT = melt(granuleCountWT, id.vars = "imageID")

In [ ]:
ggplot(granuleCountWT, aes(x = variable, y = value)) + 
    geom_violin(fill = "grey80", draw_quantiles = c(0.25, 0.5, 0.75)) + 
    xlab("") + ylab("Granule count") + 
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
ggplot(granuleCountWT, aes(x = variable, y = value, color = imageID, group = imageID)) + 
    geom_line(alpha = 0.3) + xlab("") + ylab("Granule count") + 
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
# Display overlay for each drug
for (drug in levels(subNK$Drug)){
    granuleCount = subNK[subNK$Drug == drug, grep("Count_PerfGranules", names(subNK))]
    granuleCount$imageID <- as.numeric(rownames(granuleCount))
    granuleCount = melt(granuleCount, id.vars = "imageID")
    gp <- ggplot(granuleCount, aes(x = variable, y = value)) + 
            geom_violin(fill = "grey80", draw_quantiles = c(0.25, 0.5, 0.75)) + 
            geom_line(aes(color = imageID, group = imageID), alpha = 0.15)+
            xlab("") + ylab("Granule count") + ggtitle(drug) +
            theme(axis.text.x = element_text(angle = 45, hjust = 1))
    print(gp)
    
    if(!TEST_MODE){
        ggsave(filename = paste0("Fig/NK_Drug_PerfGranuleCount_", drug, ".pdf"), plot = gp)
    }
}

## Individual granule measurements

In [ ]:
distDF = data.frame()

In [ ]:
imgWT = subNK$ImageNumber[subNK$Drug == "WT"]

In [ ]:
for (i in as.character(1:8)){
    gran = read.csv(paste0("Rsc/compiled_NKDrug_PerfPosition_PerfGranules", i, ".csv"), header=T)
    granToKeep = gran$ImageNumber %in% imgWT
    distDF = rbind(distDF, data.frame(Dist = gran[granToKeep,]$Distance_Minimum_FilterCytoplasm, Z = i))
}
distDF$Z = factor(distDF$Z, sort(levels(distDF$Z)))

In [ ]:
ggplot(distDF, aes(x = Z, y = Dist)) + geom_violin(fill = "grey80", draw_quantiles = c(0.25, 0.5, 0.75))

### Aggregate results per cell

In [ ]:
gran = read.csv("Rsc/compiled_NKDrug_PerfPosition_PerfGranules2.csv", header=T)

In [ ]:
granToKeep = gran$ImageNumber %in% imgWT
distDF = rbind(distDF, data.frame(Dist = gran[granToKeep,]$Distance_Minimum_FilterCytoplasm, Z = "1"))

We want to include dispersion both on the X and Y axes in a symmetric manner (as these directions do not represent anything particular to the cells).  
We use the Standard Distance Deviation defined as:  
$$
\begin{aligned}
SDD &= \sqrt{\frac{\sum_i^n (x_i - \hat{x})^2}{n-2} + \frac{\sum_i^n (y_i - \hat{y})^2}{n-2}} \\ 
&= \sqrt{\frac{n-1}{n-2} \times \bigg(\frac{\sum_i^n (x_i - \hat{x})^2}{n-1} + \frac{\sum_i^n (y_i - \hat{y})^2}{n-1}\bigg)} \\
&= \sqrt{\frac{n-1}{n-2} \times \big(\text{var}(x) + \text{var}(y)\big)}
\end{aligned}
$$.

In [ ]:
startTime <- proc.time()
colGran = names(gran) %in% c("Parent_FilterCytoplasm", "ImageNumber", "Location_Center_X", "Location_Center_Y")
aggGran = gran[granToKeep,] %>% 
          group_by(ImageNumber, Parent_FilterCytoplasm) %>% 
          summarise(count = n(), LocX = median(Location_Center_X), LocY = median(Location_Center_Y),
                    VarX = var(Location_Center_X), VarY = var(Location_Center_Y))
aggGran$SDD = sqrt( (aggGran$count - 1)/(aggGran$count - 2) * (aggGran$VarX + aggGran$VarY) )
proc.time() - startTime

In [ ]:
ggplot(aggGran) + geom_histogram(aes(x = SDD), binwidth = 1)